In [95]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize

In [97]:
tickers = ['MSFT', 'AMZN', 'FB', 'NVDA', 'AAPL', 'TSLA']

In [98]:
def get_stock_returns(tickers, interval, time_period):
    '''
    Valid intervals: 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo
    '''
    stock_returns = list()
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        print('Retrieve stock price for: ', ticker)
        stock_returns.append(stock.history(interval=interval, period=time_period).Close.dropna().pct_change().fillna(0))
    return np.array(stock_returns)

def data_process(returns, normalizing_factor=252):
    '''
    Normalizing factor by default adjusts to the annualized data
    '''
    covariance_matrix = np.cov(returns) * normalizing_factor
    expected_returns = np.mean(returns, axis=1) * normalizing_factor
    return covariance_matrix, expected_returns

In [99]:
stock_returns = get_stock_returns(tickers, interval='1d', time_period='1y')
Q, expected_returns = data_process(stock_returns, normalizing_factor=1)

Retrieve stock price for:  MSFT
Retrieve stock price for:  AMZN
Retrieve stock price for:  FB
Retrieve stock price for:  NVDA
Retrieve stock price for:  AAPL
Retrieve stock price for:  TSLA


In [100]:
def objective(w):
    return -1 * (np.dot(expected_returns, w) / np.dot(np.dot(w, Q), w))

In [101]:
def constraint_1(w):
    return w.sum() - 1.0

In [104]:
w0 = np.random.uniform(high=1 / len(tickers), size=len(tickers))
bounds = ([(0.01, 0.5)] * len(tickers))
cons = [{'type': 'eq', 'fun': constraint_1}]